In [ ]:
!pip install pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
# Importing the modules

from pyomo.environ import *
import pandas as pd

#### Parameters

<!--

-->

In [ ]:
# ===========================================
#           Topology Information            #
# ===========================================

total_nodes = 3 # [0, 1, 2, 3]

# CPU (Core),

total_resources = 1
resource_capacity = {'CPU' : 16,}


# ===================================
#       Functions Information       #
# ===================================

# (0)NAT (1)FW (2)TM (3)WOC (4)VOC (5)IDS
# =======================================
total_functions = 5


# Set of Functions
# id = Function Identifier
# cf = Each function require X CPU core
# bf = Traffic Processin Capacity (Function) in Mb/s (200Mb/s = 200000Kb/s)

#            (id, cf, bf)
functions = [(0, 4, 200000),
             (1, 4, 200000),
             (2, 4, 200000),
             (3, 4, 200000),
             (4, 4, 200000),
             (5, 4, 200000)]


# =============================
#     Services Information    #
# =============================

# wb = Web Service, vip = VoIP, vs = Video Streaming, og = Online Game
#      ===========        ====       ===============       ===========


# [function requested, , Bandwidth (kbps)]

services = {'wb' : {    'function' : 1,
                        'bandwidth' : 100, },
# ----------------------------------------------
            'vip' : {   'function' : 0,
                        'bandwidth' : 64, },
# ----------------------------------------------
            'vs' : {    'function' : 4,
                        'bandwidth' : 100, },
# ----------------------------------------------
            'og' : {    'function' : 5,
                        'bandwidth' : 50, },
# ----------------------------------------------
            }

# =======================================
#           Demand Information          #
# =======================================

total_demands = 3 # Total de n + 1 demandas

# Set of demands
# Demand Identifier, Service
demands = [ {     'id' : 1,
                  'service' : services['wb']},
# ----------------------------------------------
            {     'id' : 2,
                  'service' : services['vip']},
# ----------------------------------------------
            {     'id' : 3,
                  'service' : services['vs']},
# ----------------------------------------------
            {     'id' : 4,
                  'service' : services['og']},
# ----------------------------------------------
            ]

In [ ]:
import random

# Your existing data
services = {
    'wb': {'function': 1, 'bandwidth': 100,},
    'vip': {'function': 0, 'bandwidth': 64,},
    'vs': {'function': 4, 'bandwidth': 100,},
    'og': {'function': 5, 'bandwidth': 50,},
}

# List of service keys for random selection
service_keys = list(services.keys())

# Container for all demand sets
all_demand_sets = []

# Generate 30 sets
for i in range(30):
    # Container for this set of demands
    demand_set = []

    # Generate 1000 demands
    for j in range(1, 1001):
        # Randomly select a service
        service_key = random.choice(service_keys)

        # Create the demand
        demand = {'id': j, 'service': services[service_key]}

        # Add the demand to the set
        demand_set.append(demand)

    # Add the set to the list of all sets
    all_demand_sets.append(demand_set)

In [ ]:
# Print the first 5 demands of each set
for i, demand_set in enumerate(all_demand_sets):
    print(f"Set {i+1}:")
    for demand in demand_set[:5]:
        print(demand)
    print("\n")


Set 1:
{'id': 1, 'service': {'function': 1, 'bandwidth': 100}}
{'id': 2, 'service': {'function': 4, 'bandwidth': 100}}
{'id': 3, 'service': {'function': 5, 'bandwidth': 50}}
{'id': 4, 'service': {'function': 4, 'bandwidth': 100}}
{'id': 5, 'service': {'function': 4, 'bandwidth': 100}}


Set 2:
{'id': 1, 'service': {'function': 5, 'bandwidth': 50}}
{'id': 2, 'service': {'function': 4, 'bandwidth': 100}}
{'id': 3, 'service': {'function': 5, 'bandwidth': 50}}
{'id': 4, 'service': {'function': 1, 'bandwidth': 100}}
{'id': 5, 'service': {'function': 1, 'bandwidth': 100}}


Set 3:
{'id': 1, 'service': {'function': 5, 'bandwidth': 50}}
{'id': 2, 'service': {'function': 0, 'bandwidth': 64}}
{'id': 3, 'service': {'function': 5, 'bandwidth': 50}}
{'id': 4, 'service': {'function': 5, 'bandwidth': 50}}
{'id': 5, 'service': {'function': 1, 'bandwidth': 100}}


Set 4:
{'id': 1, 'service': {'function': 5, 'bandwidth': 50}}
{'id': 2, 'service': {'function': 4, 'bandwidth': 100}}
{'id': 3, 'service': {

### Modelo Load Distribution

In [ ]:
# Creating the Model
model = ConcreteModel()

# Creating the Sets
model.Nodes = RangeSet(0, total_nodes) # Set of Nodes
model.Demands = RangeSet(0, total_demands) # Set of Demands
model.Functions = RangeSet(0, total_functions) # Set od Funcitons

# Creating Variables

# allocation demand
model.d = Var(  model.Demands,
                model.Nodes,
                within = Binary,
                initialize = 0
            )

# allocation function
model.f = Var(  model.Functions,
                model.Nodes,
                within = Binary,
                initialize = 0
            )


# Objective Function : Maximize the number of demands allocated
def allocated_demands(model):

    allocated = sum( model.d[d, i]
                for d in model.Demands
                for i in model.Nodes

            )

    return allocated

model.objective = Objective(    rule = allocated_demands,
                                sense = maximize
                            )

#### Restrições

In [ ]:
def constraint_demand_allocated(model, d):
    '''
    Each demand must be allocated on a maximum of one server.
    '''
    return sum( model.d[d, s] for s in model.Nodes) <= 1


model.C1 = Constraint(  model.Demands,
                        rule = constraint_demand_allocated
                    )

In [ ]:
def constraint_demand_functions(model, d, s, f):
    '''
    A demand can only be allocated to a server that has the function it needs.
    '''
    function = demands[d]['service']['function']

    if function == f:
        return model.f[f, s] - model.d[d, s] >= 0
    else:
        return Constraint.Skip

model.C2 = Constraint(  model.Demands,
                        model.Nodes,
                        model.Functions,
                        rule = constraint_demand_functions
                    )

In [ ]:
def constraint_server_capacity(model, s):
    '''
    A server must not exceed its processing/storage caabilities.
    '''
    return sum(model.f[f, s] * functions[f][1] for f in model.Functions) <= resource_capacity['CPU']


model.C3 = Constraint(  model.Nodes,
                        rule = constraint_server_capacity
                    )

In [ ]:
def constraint_functions_capacity(model, s, f):
    '''
    A functions must not exceed its processing caabilities.
    '''

    return sum(model.d[d, s] * demands[d]['service']['bandwidth'] for d in model.Demands if (demands[d]['service']['function'] == f)) \
            <= functions[f][2] * model.f[f, s]

model.C4 = Constraint(  model.Nodes,
                        model.Functions,
                        rule = constraint_functions_capacity
                    )

#### Execução

In [ ]:
# Selecting and Creating the Solver
# ----------------------------
solver = SolverFactory('cplex')

# Calling the solver and solving the model
# ----------------------------

results = solver.solve( model,
                        #tee = True,
                        #keepfiles = True,
                    )


fitness = value(model.objective)
print(f"A função objetivo retornou o valor: {fitness}")

A função objetivo retornou o valor: 4.0


### Resultados

In [ ]:
col1 = []
for d in range(total_demands+1):
    col1.append(f"Demand {d}")

col2 = []
for s in range(total_nodes+1):
    col2.append(f"Server {s}")

allocated_demands = pd.DataFrame(columns = col2, index = list(set(col1)))

for d in range(total_demands+1):
    for s in range(total_nodes+1):
        if (value(model.d[d, s])) == 1:
            allocated_demands.loc[f"Demand {d}", f"Server {s}"] = 1
        else:
            allocated_demands.loc[f"Demand {d}", f"Server {s}"] = '.'

allocated_demands

,Server 0,Server 1,Server 2,Server 3
Demand 2,1,.,.,.
Demand 0,1,.,.,.
Demand 1,1,.,.,.
Demand 3,1,.,.,.


In [ ]:
col1 = []
for f in range(total_functions+1):
    col1.append(f"Functions {f}")

col2 = []
for s in range(total_nodes+1):
    col2.append(f"Server {s}")

allocated_functions = pd.DataFrame(columns = col2, index = list(set(col1)))
allocated_functions.sort_index(inplace = True)

for f in range(total_functions+1):
    for s in range(total_nodes+1):
        if (value(model.f[f, s])) == 1:
            allocated_functions.loc[f"Functions {f}", f"Server {s}"] = 1
        else:
            allocated_functions.loc[f"Functions {f}", f"Server {s}"] = '.'

allocated_functions

,Server 0,Server 1,Server 2,Server 3
Functions 0,1,1,1,1
Functions 1,.,.,.,.
Functions 2,1,1,1,1
Functions 3,.,.,.,.
Functions 4,.,.,.,.
Functions 5,1,1,1,1
